<a href="https://colab.research.google.com/github/JTStephens18/XCubeImplementation/blob/main/XCube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
! pip install torchinfo

In [185]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

# Binvox Processor

In [3]:
#  Copyright (C) 2012 Daniel Maturana
#  This file is part of binvox-rw-py.
#
#  binvox-rw-py is free software: you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation, either version 3 of the License, or
#  (at your option) any later version.
#
#  binvox-rw-py is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with binvox-rw-py. If not, see <http://www.gnu.org/licenses/>.
#

"""
Binvox to Numpy and back.


>>> import numpy as np
>>> import binvox_rw
>>> with open('chair.binvox', 'rb') as f:
...     m1 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims
[32, 32, 32]
>>> m1.scale
41.133000000000003
>>> m1.translate
[0.0, 0.0, 0.0]
>>> with open('chair_out.binvox', 'wb') as f:
...     m1.write(f)
...
>>> with open('chair_out.binvox', 'rb') as f:
...     m2 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims==m2.dims
True
>>> m1.scale==m2.scale
True
>>> m1.translate==m2.translate
True
>>> np.all(m1.data==m2.data)
True

>>> with open('chair.binvox', 'rb') as f:
...     md = binvox_rw.read_as_3d_array(f)
...
>>> with open('chair.binvox', 'rb') as f:
...     ms = binvox_rw.read_as_coord_array(f)
...
>>> data_ds = binvox_rw.dense_to_sparse(md.data)
>>> data_sd = binvox_rw.sparse_to_dense(ms.data, 32)
>>> np.all(data_sd==md.data)
True
>>> # the ordering of elements returned by numpy.nonzero changes with axis
>>> # ordering, so to compare for equality we first lexically sort the voxels.
>>> np.all(ms.data[:, np.lexsort(ms.data)] == data_ds[:, np.lexsort(data_ds)])
True
"""

import numpy as np

class Voxels(object):
    """ Holds a binvox model.
    data is either a three-dimensional numpy boolean array (dense representation)
    or a two-dimensional numpy float array (coordinate representation).

    dims, translate and scale are the model metadata.

    dims are the voxel dimensions, e.g. [32, 32, 32] for a 32x32x32 model.

    scale and translate relate the voxels to the original model coordinates.

    To translate voxel coordinates i, j, k to original coordinates x, y, z:

    x_n = (i+.5)/dims[0]
    y_n = (j+.5)/dims[1]
    z_n = (k+.5)/dims[2]
    x = scale*x_n + translate[0]
    y = scale*y_n + translate[1]
    z = scale*z_n + translate[2]

    """

    def __init__(self, data, dims, translate, scale, axis_order):
        self.data = data
        self.dims = dims
        self.translate = translate
        self.scale = scale
        assert (axis_order in ('xzy', 'xyz'))
        self.axis_order = axis_order

    def clone(self):
        data = self.data.copy()
        dims = self.dims[:]
        translate = self.translate[:]
        return Voxels(data, dims, translate, self.scale, self.axis_order)

    def write(self, fp):
        write(self, fp)

def read_header(fp):
    """ Read binvox header. Mostly meant for internal use.
    """
    line = fp.readline().strip()
    if not line.startswith(b'#binvox'):
        raise IOError('Not a binvox file')
    dims = list(map(int, fp.readline().strip().split(b' ')[1:]))
    translate = list(map(float, fp.readline().strip().split(b' ')[1:]))
    scale = list(map(float, fp.readline().strip().split(b' ')[1:]))[0]
    line = fp.readline()
    return dims, translate, scale

def read_as_3d_array(fp, fix_coords=True):
    """ Read binary binvox format as array.

    Returns the model with accompanying metadata.

    Voxels are stored in a three-dimensional numpy array, which is simple and
    direct, but may use a lot of memory for large models. (Storage requirements
    are 8*(d^3) bytes, where d is the dimensions of the binvox model. Numpy
    boolean arrays use a byte per element).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)
    # if just using reshape() on the raw data:
    # indexing the array as array[i,j,k], the indices map into the
    # coords as:
    # i -> x
    # j -> z
    # k -> y
    # if fix_coords is true, then data is rearranged so that
    # mapping is
    # i -> x
    # j -> y
    # k -> z
    values, counts = raw_data[::2], raw_data[1::2]
    data = np.repeat(values, counts).astype(np.bool)
    data = data.reshape(dims)
    if fix_coords:
        # xzy to xyz TODO the right thing
        data = np.transpose(data, (0, 2, 1))
        axis_order = 'xyz'
    else:
        axis_order = 'xzy'
    return Voxels(data, dims, translate, scale, axis_order)

def read_as_coord_array(fp, fix_coords=True):
    """ Read binary binvox format as coordinates.

    Returns binvox model with voxels in a "coordinate" representation, i.e.  an
    3 x N array where N is the number of nonzero voxels. Each column
    corresponds to a nonzero voxel and the 3 rows are the (x, z, y) coordinates
    of the voxel.  (The odd ordering is due to the way binvox format lays out
    data).  Note that coordinates refer to the binvox voxels, without any
    scaling or translation.

    Use this to save memory if your model is very sparse (mostly empty).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)

    values, counts = raw_data[::2], raw_data[1::2]

    sz = np.prod(dims)
    index, end_index = 0, 0
    end_indices = np.cumsum(counts)
    indices = np.concatenate(([0], end_indices[:-1])).astype(end_indices.dtype)

    values = values.astype(np.bool)
    indices = indices[values]
    end_indices = end_indices[values]

    nz_voxels = []
    for index, end_index in zip(indices, end_indices):
        nz_voxels.extend(range(index, end_index))
    nz_voxels = np.array(nz_voxels)
    # TODO are these dims correct?
    # according to docs,
    # index = x * wxh + z * width + y; // wxh = width * height = d * d

    x = nz_voxels / (dims[0]*dims[1])
    zwpy = nz_voxels % (dims[0]*dims[1]) # z*w + y
    z = zwpy / dims[0]
    y = zwpy % dims[0]
    if fix_coords:
        data = np.vstack((x, y, z))
        axis_order = 'xyz'
    else:
        data = np.vstack((x, z, y))
        axis_order = 'xzy'

    #return Voxels(data, dims, translate, scale, axis_order)
    return Voxels(np.ascontiguousarray(data), dims, translate, scale, axis_order)

def dense_to_sparse(voxel_data, dtype=np.int):
    """ From dense representation to sparse (coordinate) representation.
    No coordinate reordering.
    """
    if voxel_data.ndim!=3:
        raise ValueError('voxel_data is wrong shape; should be 3D array.')
    return np.asarray(np.nonzero(voxel_data), dtype)

def sparse_to_dense(voxel_data, dims, dtype=np.bool):
    if voxel_data.ndim!=2 or voxel_data.shape[0]!=3:
        raise ValueError('voxel_data is wrong shape; should be 3xN array.')
    if np.isscalar(dims):
        dims = [dims]*3
    dims = np.atleast_2d(dims).T
    # truncate to integers
    xyz = voxel_data.astype(np.int)
    # discard voxels that fall outside dims
    valid_ix = ~np.any((xyz < 0) | (xyz >= dims), 0)
    xyz = xyz[:,valid_ix]
    out = np.zeros(dims.flatten(), dtype=dtype)
    out[tuple(xyz)] = True
    return out

#def get_linear_index(x, y, z, dims):
    #""" Assuming xzy order. (y increasing fastest.
    #TODO ensure this is right when dims are not all same
    #"""
    #return x*(dims[1]*dims[2]) + z*dims[1] + y

def write(voxel_model, fp):
    """ Write binary binvox format.

    Note that when saving a model in sparse (coordinate) format, it is first
    converted to dense format.

    Doesn't check if the model is 'sane'.

    """
    if voxel_model.data.ndim==2:
        # TODO avoid conversion to dense
        dense_voxel_data = sparse_to_dense(voxel_model.data, voxel_model.dims)
    else:
        dense_voxel_data = voxel_model.data

    fp.write('#binvox 1\n')
    fp.write('dim '+' '.join(map(str, voxel_model.dims))+'\n')
    fp.write('translate '+' '.join(map(str, voxel_model.translate))+'\n')
    fp.write('scale '+str(voxel_model.scale)+'\n')
    fp.write('data\n')
    if not voxel_model.axis_order in ('xzy', 'xyz'):
        raise ValueError('Unsupported voxel model axis order')

    if voxel_model.axis_order=='xzy':
        voxels_flat = dense_voxel_data.flatten()
    elif voxel_model.axis_order=='xyz':
        voxels_flat = np.transpose(dense_voxel_data, (0, 2, 1)).flatten()

    # keep a sort of state machine for writing run length encoding
    state = voxels_flat[0]
    ctr = 0
    for c in voxels_flat:
        if c==state:
            ctr += 1
            # if ctr hits max, dump
            if ctr==255:
                fp.write(chr(state))
                fp.write(chr(ctr))
                ctr = 0
        else:
            # if switch state, dump
            fp.write(chr(state))
            fp.write(chr(ctr))
            state = c
            ctr = 1
    # flush out remainders
    if ctr > 0:
        fp.write(chr(state))
        fp.write(chr(ctr))


    import doctest
    doctest.testmod()

<ipython-input-3-ef32016758ed>:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def dense_to_sparse(voxel_data, dtype=np.int):
<ipython-input-3-ef32016758ed>:211: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def sparse_to_de

# Experiments

In [59]:
with open('model_normalized.solid.binvox', 'rb') as f:
  model = read_as_3d_array(f)

print(model.dims)

# print(model.data)

def convert_boolean(array):
  result_arr = array.astype(float)
  return result_arr

newModel = convert_boolean(model.data)
print(newModel[0][98])
print(newModel.shape)

[128, 128, 128]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
(128, 128, 128)


<ipython-input-3-ef32016758ed>:143: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.repeat(values, counts).astype(np.bool)


In [206]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [207]:
class Encoder(nn.Module):
  def __init__(self, in_size, trg_size):
    super(Encoder, self).__init__()

  # Initial downsampling layer to convert 128^3 into 16^3
    self.conv_layer_1 = self._conv_layer_set(1, 1, 4, 2, 1, maxPool=True)
    # self.conv_layer_2 = self._conv_layer_set(1,1,1,2,0)
    # self.conv_layer_2 = self._conv_layer_set(trg_size*8, trg_size*4, 4, 2, 1)
    # self.conv_layer_3 = self._conv_layer_set(trg_size*4, trg_size*2, 4, 2, 1)
    # self.conv_layer_4 = self._conv_layer_set(trg_size*2, trg_size, 3, 2, 1)

# Set of convolutions to convert bottleneck tensor into latent tensor of the same shape
    self.latent_conv_1 = self._conv_transpose_layer_set(1, 64, 4, 2, 1)
    self.latent_conv_2 = self._conv_transpose_layer_set(64, 128, 4, 2, 1)
    self.latent_conv_3 = self._conv_layer_set(128, 64, 4, 2, 1, maxPool=False)
    self.latent_conv_4 = self._conv_layer_set(64, 1, 4, 2, 1, maxPool=False)
    self.flatten = nn.Flatten()

  # 16^3 = 4096
    self.z_mean = nn.Linear(4096, 3)
    self.z_log_var = nn.Linear(4096, 3)



  def _conv_layer_set(self, in_c, out_c, kernel_size, stride, padding, maxPool):
    if(maxPool == True):
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding),
          nn.MaxPool3d(1, kernel_size)
      )
    else:
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding)
      )
    return conv_layer


  def _conv_transpose_layer_set(self, in_c, out_c, kernel_size, stride, padding):
    conv_layer = nn.Sequential(
        nn.ConvTranspose3d(in_c, out_c, kernel_size, stride, padding)
    )
    return conv_layer


  def reparameterize(self, z_mu, z_log_var):
    eps = torch.randn(z_mu.size(0), z_mu.size(1)).to(device)
    z = z_mu + eps * torch.exp(z_log_var/2.)
    return z


  def forward(self, x):
    # bn - bottleneck
    bn = self.conv_layer_1(x)

    out = self.latent_conv_1(bn)
    out = self.latent_conv_2(out)
    out = self.latent_conv_3(out)
    out = self.latent_conv_4(out)
    latent_vec = self.flatten(out)
    z_mean, z_log_var = self.z_mean(latent_vec), self.z_log_var(latent_vec)
    z = self.reparameterize(z_mean, z_log_var)
    return out, z_mean, z_log_var, z

In [209]:
encoder = Encoder(128, 16)
input = torch.from_numpy(newModel).unsqueeze(0).float()
print(input.shape)
# input = input.float()
output, z_mean, z_log_var, z = encoder(input)

torch.Size([1, 128, 128, 128])


In [212]:
print(output.shape)
print(z_mean)
print(z_log_var)
print(z)

torch.Size([1, 16, 16, 16])
tensor([[ 0.0122, -0.0006,  0.0202]], grad_fn=<AddmmBackward0>)
tensor([[-0.0147,  0.0037,  0.0062]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0261, -1.5908, -1.3901]], grad_fn=<AddBackward0>)


In [196]:
summary(encoder, input_size=(1, 128, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [1, 16, 16, 16]           --
├─Sequential: 1-1                        [1, 16, 16, 16]           --
│    └─Conv3d: 2-1                       [1, 64, 64, 64]           65
│    └─MaxPool3d: 2-2                    [1, 16, 16, 16]           --
├─Sequential: 1-2                        [64, 32, 32, 32]          --
│    └─ConvTranspose3d: 2-3              [64, 32, 32, 32]          4,160
├─Sequential: 1-3                        [128, 64, 64, 64]         --
│    └─ConvTranspose3d: 2-4              [128, 64, 64, 64]         524,416
├─Sequential: 1-4                        [64, 32, 32, 32]          --
│    └─Conv3d: 2-5                       [64, 32, 32, 32]          524,352
├─Sequential: 1-5                        [1, 16, 16, 16]           --
│    └─Conv3d: 2-6                       [1, 16, 16, 16]           4,097
Total params: 1,057,090
Trainable params: 1,057,090
Non-trainable par

In [197]:
print(output[0][10])

tensor([[-0.0070, -0.0094, -0.0096, -0.0095, -0.0095, -0.0095, -0.0095, -0.0095,
         -0.0095, -0.0097, -0.0096, -0.0095, -0.0095, -0.0095, -0.0095, -0.0117],
        [-0.0072, -0.0090, -0.0095, -0.0095, -0.0095, -0.0095, -0.0095, -0.0093,
         -0.0091, -0.0098, -0.0096, -0.0094, -0.0095, -0.0095, -0.0095, -0.0121],
        [-0.0071, -0.0090, -0.0095, -0.0095, -0.0095, -0.0095, -0.0095, -0.0093,
         -0.0092, -0.0095, -0.0097, -0.0095, -0.0095, -0.0095, -0.0095, -0.0121],
        [-0.0071, -0.0091, -0.0095, -0.0095, -0.0095, -0.0095, -0.0095, -0.0093,
         -0.0092, -0.0094, -0.0098, -0.0095, -0.0095, -0.0095, -0.0095, -0.0121],
        [-0.0071, -0.0091, -0.0095, -0.0095, -0.0095, -0.0095, -0.0095, -0.0094,
         -0.0093, -0.0093, -0.0098, -0.0095, -0.0095, -0.0095, -0.0095, -0.0121],
        [-0.0072, -0.0091, -0.0092, -0.0093, -0.0095, -0.0095, -0.0095, -0.0094,
         -0.0094, -0.0094, -0.0097, -0.0095, -0.0095, -0.0095, -0.0095, -0.0121],
        [-0.0071, -0.0